Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Simple RAG with Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform google-cloud-discoveryengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.2 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.

In [4]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [5]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [6]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Vertex AI Search end point URL

In [7]:
#SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/engines/daou-search-20240820_1724026123314/servingConfigs/default_search:search"


In [8]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()

## Search contexts from Vertex AI Search

### REST API Call to retrieve contexts from Vertex AI Search

In [9]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{  # https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/ContentSearchSpec
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text

### Parsing a document come from Vertex AI Search

In [10]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']
            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['content'] = chunk['content']

            # Chunks appear starting from those closest to the current Contents.
            try:
              if 'chunkMetadata' in chunk:
                if 'previousChunks' in chunk['chunkMetadata']:
                  for p_chunk in chunk['chunkMetadata']['previousChunks']:
                      item['content'] = p_chunk['content'] +"\n"+ item['content']

                if 'nextChunks' in chunk['chunkMetadata']:
                  for n_chunk in chunk['chunkMetadata']['nextChunks']:
                      item['content'] = item['content'] +"\n"+ n_chunk['content']
            except KeyError:
                pass

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

### Search context from Vertex AI Search

In [11]:
question = "스콜성 폭우 관련 기사에 대해서 요약해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)

{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/daou-search-ds-20240821_1724026314068/branches/0/documents/bf6a262ad87d1c9e80d46e21722e5e46/chunks/c1",
        "id": "c1",
        "content": "소비 심리 악화에...한섬, 2분기 영업이익 29.5%↓ 현대백화점그룹 패션전문기업 한섬은 올 2분기 매출 3417억원, 영업이익 41억원으로 전년 같은 기간 대비 각각 1.2%, 29.5% 감소했다고 6일 공시했다. 올 상반기 누적 매출은 7353억원으로 1년 전보다 2美서 한국 인디브랜드 잘 나가네...비건 뷰티 브랜드 풀리 아마존 프라임데이서 '흥행' 프리미엄 비건 뷰티 브랜드 풀리(FULLY)는 대표 제품 '그린 토마토 클레이 팩 클렌저'가 미국 아마존 프라임데이 행사에서 매출을 초과 달성하며 좋은 성과를 냈다고 6일 밝혔다. 아마존 프라임데이는 세계 최대 전자아모레퍼시픽그룹, 2분기 영업익 122억원...전년비 4.2%↑ 아모레퍼시픽그룹은 올 2분기1조57억원의 매출과 122억원의 영업이익을 기록했다고 6일 공시했다. 이는 전년 동기 대비 매출은 2.4% 하락했으며 영업이익은 4.2% 증가한 수치다. 올 2분기 아모레퍼시픽그룹은 국타가, 베트남 본격 공략...태국 등 동남아 지역도 하반기 진출 비케이브로스는 비건 화장품 브랜드 타가가 8월 중순 베트남 '쇼피(Shopee)' 입점을 시작으로 동남아 영유아 화장품 시장 공략에 나선다고 6일 밝혔다. 쇼피는 동남아 지역 최대 이커머스 플랫폼으로 베트남을 비롯해스콜성 폭우 급습에...'접는 양우산' 특수 오락가락한 날씨 대비 목적 편의점 양말 매출도 39% 쑥 백화점 '웨더웨어' 판매 급증 최근 비가 짧은 시간 

In [12]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': 'document-page0', 'uri': 'gs://daou_search_20240821/document-page0.pdf', 'content': '소비 심리 악화에...한섬, 2분기 영업이익 29.5%↓ 현대백화점그룹 패션전문기업 한섬은 올 2분기 매출 3417억원, 영업이익 41억원으로 전년 같은 기간 대비 각각 1.2%, 29.5% 감소했다고 6일 공시했다. 올 상반기 누적 매출은 7353억원으로 1년 전보다 2美서 한국 인디브랜드 잘 나가네...비건 뷰티 브랜드 풀리 아마존 프라임데이서 \'흥행\' 프리미엄 비건 뷰티 브랜드 풀리(FULLY)는 대표 제품 \'그린 토마토 클레이 팩 클렌저\'가 미국 아마존 프라임데이 행사에서 매출을 초과 달성하며 좋은 성과를 냈다고 6일 밝혔다. 아마존 프라임데이는 세계 최대 전자아모레퍼시픽그룹, 2분기 영업익 122억원...전년비 4.2%↑ 아모레퍼시픽그룹은 올 2분기1조57억원의 매출과 122억원의 영업이익을 기록했다고 6일 공시했다. 이는 전년 동기 대비 매출은 2.4% 하락했으며 영업이익은 4.2% 증가한 수치다. 올 2분기 아모레퍼시픽그룹은 국타가, 베트남 본격 공략...태국 등 동남아 지역도 하반기 진출 비케이브로스는 비건 화장품 브랜드 타가가 8월 중순 베트남 \'쇼피(Shopee)\' 입점을 시작으로 동남아 영유아 화장품 시장 공략에 나선다고 6일 밝혔다. 쇼피는 동남아 지역 최대 이커머스 플랫폼으로 베트남을 비롯해스콜성 폭우 급습에...\'접는 양우산\' 특수 오락가락한 날씨 대비 목적 편의점 양말 매출도 39% 쑥 백화점 \'웨더웨어\' 판매 급증 최근 비가 짧은 시간 쏟아졌다가 다시 더워지는 스콜성 폭우가빈번해지면서 오락가락한 날씨에 모두 활용할 수 있는 아이템들이 유통풀리, 美 아마존 프라임데이서 \'대박\'...일매출 36배 초과 달성 풀리는 대표 제품 \'그린 토마토 클레이 팩 클렌저\'가 미국 아마존 프라임데이 행사에서 6월 평균 일매출 대비 36

## Reasoning result with LLM

### Function to call LLM

In [13]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

### Run example

In [14]:
from time import perf_counter

t1_start = perf_counter()

prompt = f"""

  당신은 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 2.32075014499992 seconds




최근 스콜성 폭우가 잦아지면서 오락가락한 날씨에 대비할 수 있는 아이템들이 인기를 끌고 있습니다. 특히 백화점에서는 '웨더웨어' 판매가 급증하고 있으며, 편의점에서는 양말 매출이 39%나 뛰었습니다. 또한, 짧은 시간 동안 집중적으로 비가 내리는 스콜성 폭우에 대비해 접는 양우산의 판매도 늘어났습니다. 
